In [848]:
import plotly.graph_objects as go
import os
import pandas as pd
from config import *

In [849]:
timestamp = "20250115_02-02-09"

latest_run = True

In [850]:
# List all folders in the directory
runs = [f for f in os.listdir("csv") if os.path.isdir(os.path.join("csv", f))]

if latest_run:
    timestamp = sorted(runs)[-1]

df_game_data = pd.read_csv(f"csv/{timestamp}/game_data.csv", delimiter=";")
df_bid_logs = pd.read_csv(f"csv/{timestamp}/bid_logs.csv", delimiter=";")

df_x_demand = pd.read_csv(f"csv/{timestamp}/x_demand.csv", delimiter=";")
df_x_bought = pd.read_csv(f"csv/{timestamp}/x_bought.csv", delimiter=";")
df_x_sold = pd.read_csv(f"csv/{timestamp}/x_sold.csv", delimiter=";")
df_x_re_capacity = pd.read_csv(f"csv/{timestamp}/x_re_cap.csv", delimiter=";")
df_x_re_gen = pd.read_csv(f"csv/{timestamp}/x_re_gen.csv", delimiter=";")
df_x_th_gen = pd.read_csv(f"csv/{timestamp}/x_th_gen.csv", delimiter=";")

df_ask_prices = pd.read_csv(f"csv/{timestamp}/ask_prices.csv", delimiter=";")
df_bid_prices = pd.read_csv(f"csv/{timestamp}/bid_prices.csv", delimiter=";")
df_ask_agg = pd.read_csv(f"csv/{timestamp}/ask_agg.csv", delimiter=";")
df_bid_agg = pd.read_csv(f"csv/{timestamp}/bid_agg.csv", delimiter=";")
df_target_price_param = pd.read_csv(f"csv/{timestamp}/target_price_param.csv", delimiter=";")
df_limit_buy = pd.read_csv(f"csv/{timestamp}/limit_buy.csv", delimiter=";")
df_limit_sell = pd.read_csv(f"csv/{timestamp}/limit_sell.csv", delimiter=";")
df_target_asks = pd.read_csv(f"csv/{timestamp}/target_asks.csv", delimiter=";")
df_target_bids = pd.read_csv(f"csv/{timestamp}/target_bids.csv", delimiter=";")

df_market_positions = pd.read_csv(f"csv/{timestamp}/market_positions.csv", delimiter=";")
df_payoffs = pd.read_csv(f"csv/{timestamp}/payoffs.csv", delimiter=";")
df_revenues = pd.read_csv(f"csv/{timestamp}/revenues.csv", delimiter=";")
df_expenses = pd.read_csv(f"csv/{timestamp}/expenses.csv", delimiter=";")
df_prod_costs = pd.read_csv(f"csv/{timestamp}/prod_costs.csv", delimiter=";")
df_penalty_imbalances = pd.read_csv(f"csv/{timestamp}/penalty_imbalances.csv", delimiter=";")
df_imbalances = pd.read_csv(f"csv/{timestamp}/imbalances.csv", delimiter=";")

bidder_colors = ['green', 'purple', 'orange', 'brown', 'pink', 'olive']
ask_color = 'red'
bid_color = 'blue'
match_color = 'whitesmoke'

In [851]:
print(f'Starting parameters:')
print(f'Aggressiveness ask: {start_aggressiveness_ask}')
print(f'Aggressiveness bid: {start_aggressiveness_bid}')
print(f'Aggressiveness step factor: {aggressiveness_step_factor}')
print(f'Target price param step factor: {target_price_param_step_factor}')
print(f'Bid step factor: {bid_step_factor}')

Starting parameters:
Aggressiveness ask: [-0.22929490607261493, -0.4788429694876094, -0.7975848236200189, -0.6413578085956358, -0.5784703741613149, -0.13152868476646362]
Aggressiveness bid: [-0.7397429851087606, -0.32935398451842135, -0.26983210550999215, -0.48437673120446956, -0.25011527526241717, -0.06167047461503883]
Aggressiveness step factor: [1.4852130656793237, 0.9690704141667343, 0.8368422555509027, 1.3656381184555229, 0.6410137882771039, 1.298868695688029]
Target price param step factor: [0.7506981910361772, 0.964101810176456, 0.7693306916048687, 0.6237145250266916, 0.8288220927930482, 0.8736758345827793]
Bid step factor: [3, 3, 3, 3, 3, 3]


In [852]:
df_bid_logs

,Unnamed: 0,bidder,price,volume,transaction_price,match_flag
0,1,3.0,17.663102,2000.000000,NaN,ask
1,2,2.0,17.789893,2000.000000,NaN,ask
2,3,0.0,17.648891,2000.000000,NaN,ask
3,4,1.0,15.622719,2000.000000,NaN,ask
4,5,0.0,15.634446,2000.000000,NaN,ask
...,...,...,...,...,...,...
194,195,1.0,11.167586,1.827073,NaN,ask
195,196,0.0,11.045652,429.961148,NaN,ask
196,197,0.0,10.974952,422.360978,NaN,ask
197,198,2.0,10.973155,5.924062,NaN,ask


In [853]:
### Bids

for bidder in range(n):
    fig = go.Figure()

    # Add marker for 'bid price'
    fig.add_trace(go.Scatter(
        x=df_bid_logs.loc[df_bid_logs['bidder'] == bidder].index,
        y=df_bid_logs.loc[df_bid_logs['bidder'] == bidder, 'price'],
        mode='markers',
        name=f'Price',
        line=dict(width=2, color=bidder_colors[bidder]),  # Adjust line width if necessary
        yaxis='y'
    ))
    
    # Add marker for 'transaction price'
    fig.add_trace(go.Scatter(
        x=df_bid_logs.loc[df_bid_logs['bidder'] == bidder].index,
        y=df_bid_logs.loc[df_bid_logs['bidder'] == bidder, 'transaction_price'],
        mode='markers',
        name=f'Transaction price',
        line=dict(width=2),  # Adjust line width if necessary
        yaxis='y'
    ))
    
    # Add bar for 'volume'
    fig.add_trace(go.Bar(
        x=df_bid_logs.loc[(df_bid_logs['bidder'] == bidder) & (df_bid_logs['match_flag'] == 'bid')].index,
        y=df_bid_logs.loc[(df_bid_logs['bidder'] == bidder) & (df_bid_logs['match_flag'] == 'bid'), 'volume'],
        name='Bid volume',
        width=1.5,
        yaxis='y2',
    ))
    
    # Add bar for 'volume'
    fig.add_trace(go.Bar(
        x=df_bid_logs.loc[(df_bid_logs['bidder'] == bidder) & (df_bid_logs['match_flag'] == 'ask')].index,
        y=df_bid_logs.loc[(df_bid_logs['bidder'] == bidder) & (df_bid_logs['match_flag'] == 'ask'), 'volume'],
        name='Ask volume',
        width=1.5,
        yaxis='y2',
    ))
    
    # Update layout
    fig.update_layout(
        title=f'Bidder {bidder}',
        xaxis_title='Timestep',
        yaxis=dict(
            title='Price', 
            side='left',
            overlaying='y2',
            position=0),
        yaxis2=dict(
            title='Volume',
            side='right',
            position=1,
            )
    )
    
    # Show the plot
    fig.show()

In [854]:
df_bid_logs.loc[df_bid_logs['match_flag'] == 'ask', 'price']

0      17.663102
1      17.789893
2      17.648891
3      15.622719
4      15.634446
         ...    
194    11.167586
195    11.045652
196    10.974952
197    10.973155
198    11.520005
Name: price, Length: 120, dtype: float64

In [855]:
### Bid history

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_bid_logs.loc[df_bid_logs['match_flag'] == 'bid', 'price'].index,
    y=df_bid_logs.loc[df_bid_logs['match_flag'] == 'bid', 'price'],
    mode='lines',
    name='Bids',
    line=dict(width=2)  # Adjust line width if necessary
))

fig.add_trace(go.Scatter(
    x=df_bid_logs.loc[df_bid_logs['match_flag'] == 'ask', 'price'].index,
    y=df_bid_logs.loc[df_bid_logs['match_flag'] == 'ask', 'price'],
    mode='lines',
    name='Asks',
    line=dict(width=2)  # Adjust line width if necessary
))

# Update layout
fig.update_layout(
    title='Bid history',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()

In [856]:
### Limits

fig = go.Figure()

for bidder in range(n):
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_limit_sell[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder} Ask',
        line=dict(width=2, color=bidder_colors[bidder])  # Adjust line width if necessary
    ))
    
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_limit_buy[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder} Bid',
        line=dict(width=2, color=bidder_colors[bidder], dash='dash')  # Adjust line width if necessary
    ))

# Update layout
fig.update_layout(
    title='Limits',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()

In [857]:
### Bid-Ask spread

fig = go.Figure()

# Add lines for 'bid'
fig.add_trace(go.Scatter(
    #x=bid_data['index'],
    y=df_game_data['top_bid'],
    mode='lines',
    name='Top bid',
    line=dict(width=2, color=bid_color)  # Adjust line width if necessary
))

# Add lines for 'ask'
fig.add_trace(go.Scatter(
    y=df_game_data['top_ask'],
    mode='lines',
    name='Top Ask',
    marker=dict(size=8, color=ask_color)  # Adjust point size if necessary
))

# Add lines for 'Equilibrium price'
fig.add_trace(go.Scatter(
    #x=bid_data['index'],
    y=df_game_data['equilibrium_price_estimate'],
    mode='lines',
    name='Equilibrium price',
    line=dict(width=2)  # Adjust line width if necessary
))

# Add points for 'matches'
fig.add_trace(go.Scatter(
    y=df_game_data['transaction_price'],
    mode='markers',
    name='Matches',
    marker=dict(size=8, color=match_color)  # Adjust point size if necessary
))

# Update layout
fig.update_layout(
    title='Bid-Ask spread',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()

In [858]:
df_ask_agg

,Unnamed: 0,bidder_0,bidder_1,bidder_2,bidder_3,bidder_4,bidder_5
0,1,-0.304968,-0.12990,-0.333053,-0.296797,-0.305532,-0.506683
1,2,-0.304968,-0.12990,-0.333053,-0.296797,-0.305532,-0.506683
2,3,-0.293419,-0.12990,-0.297525,-0.296797,-0.294566,-0.399250
3,4,-0.293419,-0.12990,-0.292022,-0.291779,-0.294566,-0.345034
4,5,-0.148539,-0.12990,-0.158008,-0.153668,-0.162078,-0.242044
...,...,...,...,...,...,...,...
194,195,0.009847,0.03106,0.043632,0.007906,0.094444,0.008618
195,196,0.009847,0.03106,0.059720,0.007906,0.094444,0.008618
196,197,0.009847,0.04209,0.074025,0.007906,0.094444,0.008618
197,198,0.009847,0.04209,0.074025,0.007906,0.094444,0.013815


In [859]:
### Aggressiveness

fig = go.Figure()

for bidder in range(n):
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_ask_agg[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder} Ask',
        line=dict(width=2, color=bidder_colors[bidder])  # Adjust line width if necessary
    ))
    
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_bid_agg[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder} Bid',
        line=dict(width=2, color=bidder_colors[bidder], dash='dash')  # Adjust line width if necessary
    ))

# Update layout
fig.update_layout(
    title='Aggressiveness',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()

In [860]:
### Target price param

fig = go.Figure()

for bidder in range(n):
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_target_price_param[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder} Target price param',
        line=dict(width=2, color=bidder_colors[bidder])  # Adjust line width if necessary
    ))

# Update layout
fig.update_layout(
    title='Target price param',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()

In [861]:
### Market positions

fig = go.Figure()

for bidder in range(n):
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_market_positions[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder} Position',
        line=dict(width=2, color=bidder_colors[bidder])  # Adjust line width if necessary
    ))
    
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_imbalances[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder} Imbalance',
        line=dict(width=2, color=bidder_colors[bidder], dash='dash')  # Adjust line width if necessary
    ))

# Update layout
fig.update_layout(
    title='Market positions',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()

In [862]:
### Production

fig = go.Figure()

for bidder in range(n):
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_x_re_gen[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder} RE gen',
        line=dict(width=2, color=bidder_colors[bidder])  # Adjust line width if necessary
    ))
    
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_x_th_gen[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder} Thermal gen',
        line=dict(width=2, color=bidder_colors[bidder], dash='dash')  # Adjust line width if necessary
    ))

# Update layout
fig.update_layout(
    title='Production',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()

In [863]:
### Imbalances

fig = go.Figure()

for bidder in range(n):
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_imbalances[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder}',
        line=dict(width=2, color=bidder_colors[bidder])  # Adjust line width if necessary
    ))

# Update layout
fig.update_layout(
    title='Imbalances',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()

In [864]:
### Imbalance price

fig_volatility = go.Figure()

fig_volatility.add_trace(go.Scatter
(
    x=df_game_data.index,
    y=df_game_data['imbalance_penalty_factor'],
    mode='lines',
    name='Volatility',
    line=dict(width=2)
))

fig_volatility.update_layout(title='Imbalance penalty price')
fig_volatility.show()



In [865]:
### Payoffs

fig = go.Figure()

for bidder in range(n):
    fig.add_trace(go.Scatter(
        #x=bid_data['index'],
        y=df_payoffs[f'bidder_{bidder}'],
        mode='lines',
        name=f'Bidder {bidder}',
        line=dict(width=2, color=bidder_colors[bidder])  # Adjust line width if necessary
    ))

# Update layout
fig.update_layout(
    title='Payoffs',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()



In [866]:
### Volatility

fig_volatility = go.Figure()

fig_volatility.add_trace(go.Scatter
(
    x=df_game_data.index,
    y=df_game_data['volatility'],
    mode='lines',
    name='Volatility',
    line=dict(width=2)
))

fig_volatility.update_layout(title='Volatility')
fig_volatility.show()

In [867]:
# Prices
fig_prices = go.Figure()
for i in range(0, n-1):
    fig_prices.add_trace(go.Scatter(x=df_ask_prices.index, y=df_ask_prices[f'bidder_{i}'], line=dict(dash='dash'), mode='lines', name=f'ask_bidder_{i}'))
    fig_prices.add_trace(go.Scatter(x=df_bid_prices.index, y=df_bid_prices[f'bidder_{i}'], mode='lines', name=f'bid_bidder_{i}'))
fig_prices.update_layout(title='Prices')
fig_prices.show()